#### Project Objective: 
Calculate conditional probabilities of employee departure at specific tenure milestones (weeks 1-12) for different employee cohorts grouped by location (state) and job type using a Survival Model.

#### Grouping Requirements:
By Location:
 * Calculate probabilities for the top 5 states individually
 * Calculate one combined probability for all remaining states

By Job Type:
 * Calculate probabilities for each job type

Example Calculations for Week 1:
 * P(leaving | 1 week tenure AND employee from Utah)
 * P(leaving | 1 week tenure AND employee in job_type_1)
 * Continue for all top 5 states, remaining states combined, and all job types

 Repeat this analysis for weeks 2 through 12.

#### Final Dataset Structure: Your final dataset should contain these columns:

| Week number | State 1               | State 2               | State 3               | State 4               | State 5               | Rest of states        | Job type 1            | Job type 2            | Job type 3            |
|-------------|------------------------|------------------------|------------------------|------------------------|------------------------|------------------------|------------------------|------------------------|------------------------|
| 1           | conditional prob value | conditional prob value | conditional prob value | conditional prob value | conditional prob value | conditional prob value | conditional prob value | conditional prob value | conditional prob value |


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lifelines import KaplanMeierFitter, CoxPHFitter
from lifelines.statistics import logrank_test
import warnings
warnings.filterwarnings('ignore')

In [27]:
# read in csvs
# punches_class.csv
# employee_class.csv
df_punches = pd.read_csv('../../datasets/punches_class.csv')
df_employees = pd.read_csv('../../datasets/employee_class.csv')

df_employees['TerminationDate'] = df_employees['TerminationDate'].replace(
    ['na', 'nan', 'NaN', 'nat', 'NaT', ''], pd.NA
)

# # Now isna() will catch them
# print(df_employees['TerminationDate'].isna())


print('df_punches:\n', df_punches.head(5))
print('df_employees:\n', df_employees.head(5))
print()

# print('df_punches:\n', df_punches.info())
# print()
# print('df_employees:\n', df_employees.info())

print()
print('df_punches shape:\n',df_punches.shape)
print('df_punches UID:\n',df_punches['ID'].nunique())
print()
print('df_employees shape:\n',df_employees.shape)
print('df_employees UID:\n',df_employees['ID'].nunique())
# okay that's good, the ID is unique here.

combined = pd.concat([df_punches, df_employees], ignore_index=True)

# Count unique IDs
unique_id_count = combined['ID'].nunique()

print(f"Number of unique IDs Between Both: {unique_id_count}")


df_punches:
                  ID rounded_date
0  f2339fa1ccaa654b   2025-03-31
1  f2339fa1ccaa654b   2025-04-01
2  f2339fa1ccaa654b   2025-04-02
3  f2339fa1ccaa654b   2025-04-03
4  f2339fa1ccaa654b   2025-04-04
df_employees:
     Hire_Date TerminationDate StateCode  Terminated Job_Type                ID
0  2021-10-20      2024-06-26        TX           1   type_2  46b728354f2aaf9f
1  2022-03-16      2022-07-25        TX           1   type_1  9dfbd0d7e8e5d86e
2  2023-09-07             NaN        FL           0   type_2  b80ee989ff51c1f4
3  2025-02-27      2025-08-08        TX           1   type_1  de6d259e880d2e36
4  2021-12-30      2022-09-01        FL           1   type_1  0c6e2dbeedbccd87


df_punches shape:
 (1653335, 2)
df_punches UID:
 23341

df_employees shape:
 (85084, 6)
df_employees UID:
 85084
Number of unique IDs Between Both: 86446


employee_class, yes = 1 for Terminated, and if they have an applicable TerminationDate there.

df_employees is completely accounted for with unique IDs which is good, while out of the 1.65 million rows in df_punches, there are only 23341 unique IDs. Oh wow. So then there really are only 1362 UIDs missing from the df_employeess ID column.

I almost want to jsut use a COX model on the df_employees first to see how it does, and then try to add in the 1362 UIDs since that number may even go down further due to possible employees not being terminated.

In [ ]:
# checking if the Terminated and TerminationDate colums are valid.

# Ensure TerminationDate is parsed as datetime
df_employees['TerminationDate'] = pd.to_datetime(df_employees['TerminationDate'], errors='coerce')

# Check for valid terminations: Terminated == 1 AND TerminationDate is not null
valid_terminations = df_employees[(df_employees['Terminated'] == 1) & (df_employees['TerminationDate'].notna())]

# Count how many rows meet the condition
count_valid = len(valid_terminations)
total_rows = len(df_employees)

print(f"{count_valid} out of {total_rows} rows have valid termination status and date.")

# Rows that are invalid:
# Either Terminated == 1 but TerminationDate is missing
# OR Terminated == 0 but TerminationDate is present
invalid_terminations = df_employees[
    ((df_employees['Terminated'] == 1) & (df_employees['TerminationDate'].isna())) |
    ((df_employees['Terminated'] == 0) & (df_employees['TerminationDate'].notna()))
]

print("Invalid rows:")
print(invalid_terminations)
print(f"{len(invalid_terminations)} rows have inconsistent termination status/date.")

# Okay, we have verified the TerminationDate and Termination columns line up with each other, nice.

76383 out of 85084 rows have valid termination status and date.
Invalid rows:
Empty DataFrame
Columns: [Hire_Date, TerminationDate, StateCode, Terminated, Job_Type, ID]
Index: []
0 rows have inconsistent termination status/date.
